In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

# Utils
file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
print(file_path)
# utils_path = os.path.abspath(os.path.join(file_path, '../../utils_py'))
# sys.path.append(utils_path)
# import utils_all as utils

import warnings
warnings.filterwarnings('ignore')

SEED=0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/apartin/Dropbox/work/pilot1/cell-line-drug-sensitivity/notebooks


### rescaled_combined_single_drug_growth
1. What's `SCLC` and `SCL`?
2. What's `CONCUNIT`?
3. What's `LOG_CONCENTRATION`? How come we have negative and positive values?
4. Explain `GROWTH`

### ComboDrugGrowth_Nov2017.csv
1. Does the data come only from NCI60 (ALMANAC)?
1. What's the `SCREENER` column (unique values: `FF`, `FG`, `1A`; can it be a source of batch effect)?<br>

### NCI60_CELLNAME_to_Combo.txt
`NCI60.ID`: contain the prefix NCI60, and then cell line name<br>
`CELLNAME`: contain the prefix that corresponds to the cancer type (e.g., BR, CO, LE), and then cell line name<br>
`NAME`: contain just the cell line name<br>

### Find out:
1. How many unique drugs?
2. Consider removing toxic drugs? (drugs that inhibit growth even with small dosage)

In [5]:
DATADIR = '/Users/apartin/work/jdacs/Benchmarks/Data/Pilot1'

fname_single_drug_growth = 'rescaled_combined_single_drug_growth'
# fname_drug_growth = 'combined_single_drug_growth'

fname_combo_drug_growth = 'ComboDrugGrowth_Nov2017.csv'

fname_nci60_cellname = 'NCI60_CELLNAME_to_Combo.txt'

cellmap_path = os.path.join(DATADIR, 'NCI60_CELLNAME_to_Combo.txt')

# expression_data = 'combined_rnaseq_data'
expression_data = 'combined_rnaseq_data_lincs1000'

In [6]:
OUTDIR = './save_uno'
os.makedirs(OUTDIR, exist_ok=True)

In [7]:
train_sources = ['GDSC', 'CTRP', 'ALMANAC']
test_sources = train_sources

cell_features = ['rnaseq']
drug_features = ['descriptors', 'fingerprints']

drug_median_response_min = -1
drug_median_response_max = 1
drug_lower_response = 1
drug_upper_response = -1
drug_response_span = 0

# Load single drug dose response (df1)
#### load_single_dose_response()

In [8]:
df = pd.read_csv(os.path.join(DATADIR, fname_single_drug_growth), sep='\t', engine='c',
                  na_values=['na', '-', ''],
                  dtype={'SOURCE': str, 'DRUG_ID': str,
                         'CELLNAME': str, 'CONCUNIT': str,
                         'LOG_CONCENTRATION': np.float32,
                         'EXPID': str, 'GROWTH': np.float32})
print(f'Shape {df.shape}\n')
print(df.info())
display(df[:2])

Shape (27769716, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27769716 entries, 0 to 27769715
Data columns (total 7 columns):
SOURCE               object
DRUG_ID              object
CELLNAME             object
CONCUNIT             object
LOG_CONCENTRATION    float32
EXPID                object
GROWTH               float32
dtypes: float32(2), object(5)
memory usage: 1.2+ GB
None


,SOURCE,DRUG_ID,CELLNAME,CONCUNIT,LOG_CONCENTRATION,EXPID,GROWTH
0,CCLE,CCLE.1,CCLE.1321N1,M,-8.60206,fake_exp,117.339996
1,CCLE,CCLE.1,CCLE.1321N1,M,-8.09691,fake_exp,122.000000


In [42]:
# print(df.isna().sum().sum()) # --> no NAs
# display(df.nunique())
# print(df.SOURCE.value_counts()/1000)
# df.describe()

In [10]:
# utils.plot_boxplot(x=df['SOURCE'], y=df['LOG_CONCENTRATION'],
#              figsize=(17, 4), title='Single drug',
#              outpath=os.path.join(OUTDIR, 'single_drug_logconc_boxplot.png'))

In [11]:
# utils.plot_density(col='LOG_CONCENTRATION', split_by='SOURCE', df=df,
#              figsize=(17, 4), kind='hist', title='Hist of LOG_CONCENTRATION',
#              outpath=os.path.join(OUTDIR, 'single_drug_logconc_hist.png'));

In [12]:
# utils.plot_boxplot(x=df['SOURCE'], y=df['GROWTH'],
#              figsize=(17, 4), title='Single drug',
#              outpath=os.path.join(OUTDIR, 'single_drug_growth_rescaled_boxplot.png'))

In [13]:
# utils.plot_density(col='GROWTH', split_by='SOURCE', df=df,
#              figsize=(17, 4), kind='kde', title='Density of (rescaled) GROWTH',
#              outpath=os.path.join(OUTDIR, 'single_drug_growth_rescaled_density.png'));

In [14]:
# tmp = df[df['SOURCE']=='CCLE']
# sns.jointplot(x='LOG_CONCENTRATION', y='GROWTH', data=tmp, kind='scatter', dropna=True);

In [15]:
# https://seaborn.pydata.org/generated/seaborn.jointplot.html
# Takes too long to plot!

# sns.jointplot(x='LOG_CONCENTRATION', y='GROWTH', data=df, kind='scatter', dropna=True)
# sns.jointplot(x='LOG_CONCENTRATION', y='GROWTH', data=df, kind='kde', dropna=True, color='skyblue')

In [16]:
df['DOSE'] = -df['LOG_CONCENTRATION']
df = df.rename(columns={'CELLNAME': 'CELL', 'DRUG_ID': 'DRUG', 'EXPID': 'STUDY'})
df = df[['SOURCE', 'CELL', 'DRUG', 'DOSE', 'GROWTH', 'STUDY']]
df['GROWTH'] /= 100

In [17]:
display(df[:2])

,SOURCE,CELL,DRUG,DOSE,GROWTH,STUDY
0,CCLE,CCLE.1321N1,CCLE.1,8.60206,1.1734,fake_exp
1,CCLE,CCLE.1321N1,CCLE.1,8.09691,1.2200,fake_exp


In [18]:
df = df.rename(columns={'DRUG': 'DRUG1', 'DOSE': 'DOSE1'})
df['DRUG2'] = np.nan  # we don't have the 2nd drug but that is required for the DL model
df['DOSE2'] = np.nan  # we don't have the 2nd drug but that is required for the DL model
df['DRUG2'] = df['DRUG2'].astype(object)
df['DOSE2'] = df['DOSE2'].astype(np.float32)
df = df[['SOURCE', 'CELL', 'DRUG1', 'DOSE1', 'DRUG2', 'DOSE2', 'GROWTH', 'STUDY']]

In [19]:
display(df[:2])

,SOURCE,CELL,DRUG1,DOSE1,DRUG2,DOSE2,GROWTH,STUDY
0,CCLE,CCLE.1321N1,CCLE.1,8.60206,NaN,NaN,1.1734,fake_exp
1,CCLE,CCLE.1321N1,CCLE.1,8.09691,NaN,NaN,1.2200,fake_exp


In [20]:
df1 = df.copy()
del df
print(df1.shape)
print(df1.columns.tolist())

(27769716, 8)
['SOURCE', 'CELL', 'DRUG1', 'DOSE1', 'DRUG2', 'DOSE2', 'GROWTH', 'STUDY']


In [21]:
print('Loaded {} single drug dose response measurements'.format(df1.shape[0]))

Loaded 27769716 single drug dose response measurements


In [22]:
# Unique drugs per source
# tmp = df1.groupby(['SOURCE'])['DRUG1'].agg(['nunique']).reset_index().rename(columns={'nunique': 'unq_drugs_tested'})
# display(tmp.sort_values('SOURCE'))

In [23]:
# Unique cells per source
# tmp = df1.groupby(['SOURCE'])['CELL'].agg(['nunique']).reset_index().rename(columns={'nunique': 'unq_cells_screened'})
# display(tmp.sort_values('SOURCE'))

In [24]:
tmp = df1.groupby(['SOURCE']).agg({'CELL': 'nunique', 'DRUG1': 'nunique'}).reset_index()
tmp.rename(columns={'CELL': 'unq_cells_screened', 'DRUG1': 'unq_drugs_tested'}, inplace=True)
display(tmp.sort_values('SOURCE'))

,SOURCE,unq_cells_screened,unq_drugs_tested
0,CCLE,504,24
1,CTRP,887,544
2,GDSC,1075,249
3,NCI60,59,52671
4,SCL,65,445
5,SCLC,70,526
6,gCSI,409,16


# Load drug pair (combo) dose response (df2)
#### load_combo_dose_response()

In [25]:
df = pd.read_csv(os.path.join(DATADIR, fname_combo_drug_growth), sep=',', engine='c',
                 na_values=['na','-',''],
                 usecols=['CELLNAME', 'NSC1', 'CONC1', 'NSC2', 'CONC2',
                          'PERCENTGROWTH', 'VALID', 'SCREENER', 'STUDY',
                          'PANEL'],
                 # nrows=10000,
                 dtype={'CELLNAME': str, 'NSC1': str, 'NSC2': str,
                        'CONC1': np.float32, 'CONC2': np.float32,
                        'PERCENTGROWTH':np.float32, 'VALID': str,
                        'SCREENER': str, 'STUDY': str},
                 error_bad_lines=False, warn_bad_lines=True)

# df = pd.read_csv(os.path.join(DATADIR, fname_combo_drug_growth), sep=',', engine='c',
#                  na_values=['na','-',''],
#                  usecols=['CELLNAME', 'NSC1', 'CONC1', 'NSC2', 'CONC2',
#                           'PERCENTGROWTH', 'VALID', 'SCREENER', 'STUDY'],
#                  # nrows=10000,
#                  dtype={'CELLNAME': str, 'NSC1': str, 'NSC2': str,
#                         'CONC1': np.float32, 'CONC2': np.float32,
#                         'PERCENTGROWTH':np.float32, 'VALID': str,
#                         'SCREENER': str, 'STUDY': str},
#                  error_bad_lines=False, warn_bad_lines=True)

In [26]:
print(f'Shape {df.shape}\n')
# print(df.info())
display(df[:2])

Shape (3686475, 10)



,SCREENER,STUDY,NSC1,CONC1,NSC2,CONC2,PERCENTGROWTH,VALID,PANEL,CELLNAME
0,FG,PZUT00156_86_01_T72,752,1.000000e-07,3088,0.000001,85.978996,Y,Renal Cancer,786-0
1,FG,PZUT00156_86_01_T72,752,1.000000e-07,3088,0.000010,100.903000,Y,Renal Cancer,786-0


In [27]:
print(len(df.SCREENER.value_counts()))
display(df.SCREENER.value_counts()[:3])

# print('\n\n',len(df.STUDY.value_counts()))
# display(df.STUDY.value_counts()[:3])

print('\n\n',len(df.PANEL.value_counts()))
display(df.PANEL.value_counts())

# print(len(df.TESTDATE.value_counts()))
# display(df.TESTDATE.value_counts()[:3])

# print(len(df.PLATE.value_counts()))
# display(df.PLATE.value_counts()[:3])

3


FF    2062098
FG    1415772
1A     208605
Name: SCREENER, dtype: int64



 9


Non-Small Cell Lung Cancer    554157
Melanoma                      531922
Renal Cancer                  497351
Colon Cancer                  437857
Ovarian Cancer                435512
Breast Cancer                 373161
CNS Cancer                    368334
Leukemia                      363452
Prostate Cancer               124729
Name: PANEL, dtype: int64

In [28]:
df = df[df['VALID']=='Y']
df['SOURCE'] = 'ALMANAC.' + df['SCREENER']

In [29]:
# Add the column 'CELL' that contains both the source and the cell line name (SOURCE.CELLNAME)
df_cellmap = pd.read_table(cellmap_path)
print(df_cellmap.shape)
display(df_cellmap[:2])

df_cellmap.set_index('Name', inplace=True)
cellmap = df_cellmap[['NCI60.ID']].to_dict()['NCI60.ID']
df['CELL'] = df['CELLNAME'].map(lambda x: cellmap[x])

(61, 4)


,NCI60.ID,CELLNAME,Panel,Name
0,NCI60.BT-549,BR.BT_549,Breast Cancer,BT-549
1,NCI60.HS-578T,BR.HS578T,Breast Cancer,HS 578T


In [30]:
display(df[:2])

,SCREENER,STUDY,NSC1,CONC1,NSC2,CONC2,PERCENTGROWTH,VALID,PANEL,CELLNAME,SOURCE,CELL
0,FG,PZUT00156_86_01_T72,752,1.000000e-07,3088,0.000001,85.978996,Y,Renal Cancer,786-0,ALMANAC.FG,NCI60.786-0
1,FG,PZUT00156_86_01_T72,752,1.000000e-07,3088,0.000010,100.903000,Y,Renal Cancer,786-0,ALMANAC.FG,NCI60.786-0


In [31]:
# plot_boxplot(x=df['PANEL'], y=df['CONC1'], figsize=(17, 4),
#              title='CONC1 (from combo): original',
#              outpath=os.path.join(OUTDIR, 'combo_drugs_conc1_boxplot.png'))

In [32]:
# plot_density(col='CONC1', split_by='PANEL', df=df,
#              figsize=(17, 4), kind='hist', title='Hist of CONC1 (from combo): original',
#              outpath=os.path.join(OUTDIR, 'combo_drugs_conc1_hist.png'));

In [33]:
# Rescale the concentration
# TODO: for best practices probably need to add a small value to log10(df['CONC1']+epsilon)
print(df['CONC1'].min(), df['CONC1'].max())
print(df['CONC2'].min(), df['CONC2'].max())
df['DOSE1'] = -np.log10(df['CONC1'])
df['DOSE2'] = -np.log10(df['CONC2'])

1e-13 0.01
1e-12 0.001


In [34]:
# plot_boxplot(x=df['PANEL'], y=df['DOSE1'], figsize=(17, 4),
#              title='DOSE1 (from combo): -log10(CONC1)',
#              outpath=os.path.join(OUTDIR, 'combo_drugs_log(conc1)_boxplot.png'))

In [35]:
# plot_density(col='DOSE1', split_by='PANEL', df=df,
#              figsize=(17, 4), kind='hist', title='Hist of DOSE1 (from combo): -log10(CONC1)',
#              outpath=os.path.join(OUTDIR, 'combo_drugs_log(conc1)_hist.png'));

In [36]:
print(np.sum(df['DOSE1'].isna()))
print(np.sum(df['DOSE2'].isna()))

# fig, ax = plt.subplots(figsize=(15, 6))
# for i, s in enumerate(df.PANEL.unique()):
#     # print(s)
#     tmp = df[df['PANEL']==s].copy()
#     v = tmp['DOSE2']
#     ax.hist(v[~np.isnan(v)], bins=100, label=s)
# ax.set_title('Histogram of DOSE2');
# ax.grid(True)
# # plt.savefig(fname=os.path.join(OUTDIR, 'dose1'))

0
812961


In [37]:
df['DRUG1'] = 'NSC.' + df['NSC1']
df['DRUG2'] = 'NSC.' + df['NSC2']
df['GROWTH'] = df['PERCENTGROWTH'] / 100
df = df[['SOURCE', 'CELL', 'DRUG1', 'DOSE1', 'DRUG2', 'DOSE2', 'GROWTH', 'STUDY']]

In [38]:
print('Unique NSC1 drugs: ', len(df['DRUG1'].unique()))
print('Unique NSC2 drugs: ', len(df['DRUG2'].unique()))

Unique NSC1 drugs:  105
Unique NSC2 drugs:  105


In [39]:
df2 = df.copy()
del df
print(df2.shape)
print(df2.columns.tolist())

(3686475, 8)
['SOURCE', 'CELL', 'DRUG1', 'DOSE1', 'DRUG2', 'DOSE2', 'GROWTH', 'STUDY']


In [40]:
print('Loaded {} drug pair dose response measurements'.format(df2.shape[0]))

Loaded 3686475 drug pair dose response measurements


## Merge df1 (single drug) and df2 (drug pairs)

In [36]:
# Show df1 (single drug)
print(df1.shape)
display(df1[:2])
# print(df1['SOURCE'].value_counts())

(27769716, 8)


,SOURCE,CELL,DRUG1,DOSE1,DRUG2,DOSE2,GROWTH,STUDY
0,CCLE,CCLE.1321N1,CCLE.1,8.60206,NaN,NaN,1.1734,fake_exp
1,CCLE,CCLE.1321N1,CCLE.1,8.09691,NaN,NaN,1.2200,fake_exp


In [37]:
# Show df1 (combo drugs)
print(df2.shape)
display(df2[:2])
# print(df2['SOURCE'].value_counts())

(3686475, 8)


,SOURCE,CELL,DRUG1,DOSE1,DRUG2,DOSE2,GROWTH,STUDY
0,ALMANAC.FG,NCI60.786-0,NSC.752,7.0,NSC.3088,6.0,0.85979,PZUT00156_86_01_T72
1,ALMANAC.FG,NCI60.786-0,NSC.752,7.0,NSC.3088,5.0,1.00903,PZUT00156_86_01_T72


In [38]:
df = pd.concat([df1, df2])
print('Combined dose response data contains sources: {}'.format(df['SOURCE'].unique()))

Combined dose response data contains sources: ['CCLE' 'CTRP' 'gCSI' 'GDSC' 'NCI60' 'SCL' 'SCLC' 'ALMANAC.FG'
 'ALMANAC.FF' 'ALMANAC.1A']


In [39]:
df = df.rename(columns={'SOURCE': 'Source', 'CELL': 'Sample',
                        'DRUG1': 'Drug1', 'DRUG2': 'Drug2',
                        'DOSE1': 'Dose1', 'DOSE2': 'Dose2',
                        'GROWTH': 'Growth', 'STUDY': 'Study'})

In [40]:
df_response = df.copy()
del df1, df2, df
print(df_response.shape)
print(df_response.columns.tolist())

(31456191, 8)
['Source', 'Sample', 'Drug1', 'Dose1', 'Drug2', 'Dose2', 'Growth', 'Study']


### Summarize Response Data: summarize_response_data()

In [41]:
display(df_response[:2])

,Source,Sample,Drug1,Dose1,Drug2,Dose2,Growth,Study
0,CCLE,CCLE.1321N1,CCLE.1,8.60206,NaN,NaN,1.1734,fake_exp
1,CCLE,CCLE.1321N1,CCLE.1,8.09691,NaN,NaN,1.2200,fake_exp


In [42]:
# func summarize_response_data()
# Takes long time!

# df = df_response.copy()
# df_sum = df.groupby('Source').agg({'Growth': 'count', 'Sample': 'nunique',
#                                    'Drug1': 'nunique', 'Drug2': 'nunique'})
# df_sum['MedianDose'] = df_response.groupby('Source').agg({'Dose1': 'median'})
# print(df_sum.shape)

# df_sum

**Are there really >52k drugs in NCI60 ??**

### Encode sources: encode_sources()

In [43]:
all_sources = df_response['Source'].unique()

In [44]:
# func encode_sources()
sources = all_sources.copy()

df = pd.get_dummies(sources, prefix='source', prefix_sep='.')
df['Source'] = sources
source_l1 = df['Source'].str.extract('^(\S+)\.', expand=False)
df1 = pd.get_dummies(source_l1, prefix='source.L1', prefix_sep='.')
df = pd.concat([df1, df], axis=1)
df = df.set_index('Source').reset_index()

df_source = df.copy()
del df

In [45]:
# df_source

### Find out how many unique samples and drugs

In [46]:
train_sep_sources = [x for x in all_sources for y in train_sources if x.startswith(y)]
test_sep_sources = [x for x in all_sources for y in test_sources if x.startswith(y)]

print('train_sep_sources', train_sep_sources)
print('test_sep_sources ', test_sep_sources)

train_sep_sources ['CTRP', 'GDSC', 'ALMANAC.FG', 'ALMANAC.FF', 'ALMANAC.1A']
test_sep_sources  ['CTRP', 'GDSC', 'ALMANAC.FG', 'ALMANAC.FF', 'ALMANAC.1A']


In [47]:
print(df_response[['Drug1']].shape)
print(df_response[['Drug2']].shape)

ids1 = df_response[['Drug1']].drop_duplicates().rename(columns={'Drug1':'Drug'})
ids2 = df_response[['Drug2']].drop_duplicates().rename(columns={'Drug2':'Drug'})

print(ids1.shape)
print(ids2.shape)

(31456191, 1)
(31456191, 1)
(53520, 1)
(105, 1)


In [48]:
# Get all the samples and drugs for which we have response data!!
df_drugs_with_response = pd.concat([ids1, ids2]).drop_duplicates().dropna().reset_index(drop=True)
df_cells_with_response = df_response[['Sample']].drop_duplicates().reset_index(drop=True)
print('Combined raw dose response data has {} unique samples and {} unique drugs'.format(df_cells_with_response.shape[0], df_drugs_with_response.shape[0]))

Combined raw dose response data has 3070 unique samples and 53520 unique drugs


# Select drugs with specific resposne range:
#### select_drugs_with_response_range()

In [49]:
display(df_response[:2])

,Source,Sample,Drug1,Dose1,Drug2,Dose2,Growth,Study
0,CCLE,CCLE.1321N1,CCLE.1,8.60206,NaN,NaN,1.1734,fake_exp
1,CCLE,CCLE.1321N1,CCLE.1,8.09691,NaN,NaN,1.2200,fake_exp


In [50]:
print(f'Limiting drugs to those with response min <= {drug_lower_response}, max >= {drug_upper_response}, span >= {drug_response_span}, median_min <= {drug_median_response_min}, median_max >= {drug_median_response_max}')

Limiting drugs to those with response min <= 1, max >= -1, span >= 0, median_min <= -1, median_max >= 1


In [51]:
lower = drug_lower_response
upper = drug_upper_response
span  = drug_response_span
lower_median = drug_median_response_min
upper_median = drug_median_response_max

In [52]:
df = df_response.groupby(['Drug1', 'Sample'])['Growth'].agg(['min', 'max', 'median'])
# tmp = df_response.groupby(['Drug1', 'Sample']).agg({'Growth': ['min', 'max', 'median']})  # (ap) (almost) the same result
df[:3]

min    max  median
Drug1  Sample                          
CCLE.1 CCLE.1321N1  0.14  1.220  0.9027
       CCLE.22RV1  -0.42  1.250  0.6100
       CCLE.42MGBA -0.26  1.248  0.9595

In [53]:
# When compute span, ignore growth outside the range [-1, 1]. E.g., round_to_one(1.22) - round_to_zero(0.14) = 0.86
df['span'] = df['max'].clip(lower=-1, upper=1) - df['min'].clip(lower=-1, upper=1)
df[:3]

min    max  median  span
Drug1  Sample                                
CCLE.1 CCLE.1321N1  0.14  1.220  0.9027  0.86
       CCLE.22RV1  -0.42  1.250  0.6100  1.42
       CCLE.42MGBA -0.26  1.248  0.9595  1.26

In [54]:
# For every drug, compute the mean across samples for 'min', 'max', 'median', 'span' of growth
df = df.groupby('Drug1').mean().reset_index().rename(columns={'Drug1': 'Drug'})
df[:3]

,Drug,min,max,median,span
0,CCLE.1,-0.090962,1.280473,0.899474,1.089913
1,CCLE.10,0.076899,1.107306,0.784485,0.892617
2,CCLE.11,-0.939960,1.162450,-0.423682,1.904465


In [55]:
mask = (df['min'] <= lower) & (df['max'] >= upper) & (df['span'] >= span)
mask[:3]

0    True
1    True
2    True
dtype: bool

In [56]:
if lower_median:
    mask &= (df['median'] >= lower_median)
if upper_median:
    mask &= (df['median'] <= upper_median)
df_sub = df[mask]

In [57]:
df_selected_drugs = df_sub.copy()
print(df_selected_drugs.shape)
print(df_selected_drugs.columns.tolist())
del df_sub, df

(47005, 5)
['Drug', 'min', 'max', 'median', 'span']


In [58]:
print('Selected {} drugs from {}'.format(df_selected_drugs.shape[0], df_response['Drug1'].nunique()))

Selected 47005 drugs from 53520


In [59]:
# (ap) extract selected drugs for CCLE
# df_selected_drugs[df_selected_drugs['Drug'].map(lambda s: True if 'CCLE' in s.split('.')[0] else False)]

# Extract cell features
#### load_cell_rnaseq()

In [60]:
scaling = 'std'
imputing = 'mean'

In [61]:
# Load the data
path = os.path.join(DATADIR, expression_data)
df_cols = pd.read_table(path, engine='c', nrows=0)
total = df_cols.shape[1] - 1  # remove Sample column
if 'Cancer_type_id' in df_cols.columns:
    total -= 1
usecols = None

dtype_dict = dict((x, np.float32) for x in df_cols.columns[1:])
df = pd.read_table(path, engine='c', usecols=usecols, dtype=dtype_dict)
print(df.shape)

(15196, 943)


In [62]:
# Embedding data source into features by onehot
# THE IDEA IS TO USE THIS AS REGULARIZER FOR BATCH EFFECT!!(??)
prefixes = df['Sample'].str.extract('^([^.]*)', expand=False).rename('Source')
sources = prefixes.drop_duplicates().reset_index(drop=True)
df_source = pd.get_dummies(sources, prefix='rnaseq.source', prefix_sep='.')
df_source = pd.concat([sources, df_source], axis=1)
df_source

,Source,rnaseq.source.CCLE,rnaseq.source.CTRP,rnaseq.source.GDC,rnaseq.source.GDSC,rnaseq.source.NCI60,rnaseq.source.NCIPDM,rnaseq.source.gCSI
0,CCLE,1,0,0,0,0,0,0
1,CTRP,0,1,0,0,0,0,0
2,GDC,0,0,1,0,0,0,0
3,GDSC,0,0,0,1,0,0,0
4,NCI60,0,0,0,0,1,0,0
5,NCIPDM,0,0,0,0,0,1,0
6,gCSI,0,0,0,0,0,0,1


In [63]:
# Merge 'Sample' and one-hot encoded Source
df1 = df['Sample']
df_sample_source = pd.concat([df1, prefixes], axis=1)
df1 = df_sample_source.merge(df_source, on='Source', how='left').drop('Source', axis=1)
print(f'Embedding RNAseq data source into features: {df1.shape[1]-1} additional columns')
df1[:2]

Embedding RNAseq data source into features: 7 additional columns


,Sample,rnaseq.source.CCLE,rnaseq.source.CTRP,rnaseq.source.GDC,rnaseq.source.GDSC,rnaseq.source.NCI60,rnaseq.source.NCIPDM,rnaseq.source.gCSI
0,CCLE.22RV1,1,0,0,0,0,0,0
1,CCLE.2313287,1,0,0,0,0,0,0


In [64]:
# df with gene features only
df2 = df.drop('Sample', 1)
df2 = df2.add_prefix('rnaseq.')
df2 = utils.impute_and_scale(df2, scaling, imputing)

In [65]:
df = pd.concat([df1, df2], axis=1)
print('Loaded combined RNAseq data: ', df.shape)

Loaded combined RNAseq data:  (15196, 950)


In [66]:
df_cell_rnaseq = df.copy()
print(df_cell_rnaseq.shape)
print(df_cell_rnaseq.columns.tolist()[:9])
del df, df1, df2

(15196, 950)
['Sample', 'rnaseq.source.CCLE', 'rnaseq.source.CTRP', 'rnaseq.source.GDC', 'rnaseq.source.GDSC', 'rnaseq.source.NCI60', 'rnaseq.source.NCIPDM', 'rnaseq.source.gCSI', 'rnaseq.AARS']


# Extract drug descriptors
#### load_drug_descriptors()

In [ ]:
# load_drug_info()
path = os.path.join(DATADIR, 'drug_info')
df = pd.read_table(path, dtype=object)
df['PUBCHEM'] = 'PubChem.CID.' + df['PUBCHEM']
df_info = df.copy()

df_info['Drug'] = df_info['PUBCHEM']
print(df_info.shape)
df_info[:2]

**How come we have now only 846 unique drugs?? (we had a lot more before)**

In [68]:
df_info.nunique()

ID            846
NAME          736
CLEAN_NAME    729
SMILES        402
INCHIKEY      299
PUBCHEM       516
Drug          516
dtype: int64

In [69]:
df_info['ID'].map(lambda s: s.split('.')[0]).value_counts()  # (ap)

CTRP    544
GDSC    262
CCLE     24
gCSI     16
Name: ID, dtype: int64

In [70]:
# (df_info['CLEAN_NAME']=='AEW541').sum()
# df_info['CLEAN_NAME'].nunique()
tmp = df_info.copy()
tmp['SOURCE'] = df_info['ID'].map(lambda s: s.split('.')[0])
tmp.groupby(['SOURCE']).agg({'CLEAN_NAME': 'unique'}).reset_index()

,SOURCE,CLEAN_NAME
0,CCLE,"[AEW541, ZD6474, PANOBINOSTAT, SORAFENIB, IRIN..."
1,CTRP,"[16BETABROMOANDROSTERONE, ABT263, BRDK45681478..."
2,GDSC,"[ERLOTINIB, AICARIBONUCLEOTIDE, CAMPTOTHECIN, ..."
3,gCSI,"[BID1870, LAPATINIB, MS275, PD0325901, PACLITA..."


### load_drug_set_descriptors( drug_set = 'Combined_PubChem' )

In [71]:
path = os.path.join(DATADIR, 'Combined_PubChem_dragon7_descriptors.tsv')
df_cols = pd.read_table(path, engine='c', nrows=0)
total = df_cols.shape[1] - 1

dtype_dict = dict((x, np.float32) for x in df_cols.columns[1:])
df = pd.read_table(path, engine='c', usecols=usecols, dtype=dtype_dict, na_values=['na', '-', ''])

print(df.shape)
df.iloc[:2, :10]

(517, 5271)


,NAME,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp
0,PubChem.CID.10026128,438.51001,7.831,36.275002,56.813999,37.387001,63.286999,0.648,1.015,0.668
1,PubChem.CID.10027278,460.48999,8.688,36.691002,54.224998,37.141998,59.958000,0.692,1.023,0.701


In [72]:
# Extract drug features (descriptors and fingerprints) into df2
df1 = pd.DataFrame(df.loc[:, 'NAME'])
df1.rename(columns={'NAME': 'Drug'}, inplace=True)
df2 = df.drop('NAME', 1)
df2 = utils.impute_and_scale(df2, scaling=None, imputing=None, dropna=None)

In [73]:
df = pd.concat([df1, df2], axis=1)
df_desc = df.copy()
del df, df1, df2
print(df_desc.shape)

(517, 5271)


Now merge `df_info` and `df_desc`

In [74]:
print('df_info', df_info.shape)
print('df_desc', df_desc.shape)

df_info (846, 7)
df_desc (517, 5271)


In [75]:
display(df_info[:2])
display(df_desc[:2])

,ID,NAME,CLEAN_NAME,SMILES,INCHIKEY,PUBCHEM,Drug
0,CCLE.1,AEW541,AEW541,C1CN(C1)CC2CC(C2)N3C=C(C4=C3N=CN=C4N)C5=CC(=CC...,AECDBHGVIIRMOI-UHFFFAOYSA-N,PubChem.CID.11476171,PubChem.CID.11476171
1,CCLE.10,ZD-6474,ZD6474,CN1CCC(CC1)COC2=C(C=C3C(=C2)N=CN=C3NC4=C(C=C(C...,UHTHHESEBZOYNR-UHFFFAOYSA-N,PubChem.CID.3081361,PubChem.CID.3081361


,Drug,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,...,CATS3D_10_LL,CATS3D_11_LL,CATS3D_12_LL,CATS3D_13_LL,CATS3D_14_LL,CATS3D_15_LL,CATS3D_16_LL,CATS3D_17_LL,CATS3D_18_LL,CATS3D_19_LL
0,PubChem.CID.10026128,438.51001,7.831,36.275002,56.813999,37.387001,63.286999,0.648,1.015,0.668,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PubChem.CID.10027278,460.48999,8.688,36.691002,54.224998,37.141998,59.958000,0.692,1.023,0.701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
# Important!
df_desc = pd.merge(df_info[['ID', 'Drug']], df_desc, on='Drug').drop('Drug', axis=1).rename(columns={'ID': 'Drug'})
print(df_desc.shape)
df_desc[:2]

(661, 5271)


,Drug,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,...,CATS3D_10_LL,CATS3D_11_LL,CATS3D_12_LL,CATS3D_13_LL,CATS3D_14_LL,CATS3D_15_LL,CATS3D_16_LL,CATS3D_17_LL,CATS3D_18_LL,CATS3D_19_LL
0,CCLE.1,439.609985,7.090,39.143002,61.439999,41.619999,69.682999,0.631,0.991,0.671,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CCLE.10,475.399994,8.804,34.717999,54.522999,36.597000,61.160000,0.643,1.010,0.678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### load_drug_set_descriptors( drug_set = 'NCI60' )

In [77]:
path = os.path.join(DATADIR, 'NCI60_dragon7_descriptors.tsv')
df_cols = pd.read_table(path, engine='c', nrows=0)
total = df_cols.shape[1] - 1

dtype_dict = dict((x, np.float32) for x in df_cols.columns[1:])
df = pd.read_table(path, engine='c', usecols=usecols, dtype=dtype_dict, na_values=['na', '-', ''])

print(df.shape)
df.iloc[:2, :10]

(52846, 5271)


,NAME,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp
0,NSC.1,122.129997,8.142,10.01,15.305,10.193000,16.664000,0.667,1.020,0.680
1,NSC.100044,281.309998,8.037,21.60,36.236,21.653999,40.404999,0.617,1.035,0.619


In [78]:
# Extract drug features (descriptors and fingerprints) into df2
df1 = pd.DataFrame(df.loc[:, 'NAME'])
df1.rename(columns={'NAME': 'Drug'}, inplace=True)
df2 = df.drop('NAME', 1)
df2 = utils.impute_and_scale(df2, scaling=None, imputing=None, dropna=None)

In [79]:
df = pd.concat([df1, df2], axis=1)
df_desc2 = df.copy()
del df, df1, df2
print(df_desc.shape)

(661, 5271)


Merge df_desc (Combined_PubChem) and df_desc2 (NCI60)

In [80]:
print('df_desc ', df_desc.shape)
print('df_desc2', df_desc2.shape)

df_desc  (661, 5271)
df_desc2 (52846, 5271)


In [81]:
display(df_desc[:2])
display(df_desc2[:2])

,Drug,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,...,CATS3D_10_LL,CATS3D_11_LL,CATS3D_12_LL,CATS3D_13_LL,CATS3D_14_LL,CATS3D_15_LL,CATS3D_16_LL,CATS3D_17_LL,CATS3D_18_LL,CATS3D_19_LL
0,CCLE.1,439.609985,7.090,39.143002,61.439999,41.619999,69.682999,0.631,0.991,0.671,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CCLE.10,475.399994,8.804,34.717999,54.522999,36.597000,61.160000,0.643,1.010,0.678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Drug,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,...,CATS3D_10_LL,CATS3D_11_LL,CATS3D_12_LL,CATS3D_13_LL,CATS3D_14_LL,CATS3D_15_LL,CATS3D_16_LL,CATS3D_17_LL,CATS3D_18_LL,CATS3D_19_LL
0,NSC.1,122.129997,8.142,10.01,15.305,10.193000,16.664000,0.667,1.020,0.680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NSC.100044,281.309998,8.037,21.60,36.236,21.653999,40.404999,0.617,1.035,0.619,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
df_desc = pd.concat([df_desc, df_desc2], axis=0).reset_index(drop=True)
df1 = pd.DataFrame(df_desc.loc[:, 'Drug'])
df2 = df_desc.drop('Drug', 1)
df2 = utils.impute_and_scale(df2, scaling='std', imputing='mean', dropna=None)
df2 = df2.add_prefix('dragon7.')
df_desc = pd.concat([df1, df2], axis=1)
print('Loaded combined dragon7 drug descriptors: ', df_desc.shape)

Loaded combined dragon7 drug descriptors:  (53507, 5271)


In [83]:
df_drug_desc = df_desc.copy()
del df_desc, df1, df2
print(df_drug_desc.shape)
print(df_drug_desc.columns.tolist()[:10])

(53507, 5271)
['Drug', 'dragon7.MW', 'dragon7.AMW', 'dragon7.Sv', 'dragon7.Se', 'dragon7.Sp', 'dragon7.Si', 'dragon7.Mv', 'dragon7.Me', 'dragon7.Mp']


## Extract drug fingerprints
#### load_drug_fingerprints()

In [84]:
# load_drug_fingerprints(ncols=None, scaling='std', imputing='mean', dropna=None, add_prefix=True)
df_drug_fp = utils.load_drug_fingerprints(ncols=0, scaling='std', dropna=None)

## Continue ...

In [85]:
cell_df_dict = {'rnaseq': 'df_cell_rnaseq'}
drug_df_dict = {'descriptors': 'df_drug_desc',
                'fingerprints': 'df_drug_fp'}

In [86]:
# (ap)
df_cell_ids = df_cells_with_response
print('df_cell_ids.shape', df_cell_ids.shape)
fea = cell_features[0]
tmp = locals()[cell_df_dict[fea]]
print('tmp.shape', tmp.shape)

tmp2 = df_cell_ids.merge(tmp[['Sample']], on='Sample').drop_duplicates()
print('tmp2.shape', tmp2.shape)

df_cell_ids.shape (3070, 1)
tmp.shape (15196, 950)
tmp2.shape (2373, 1)


In [87]:
df_cell_ids = df_cells_with_response
for fea in cell_features:
    df_cell = locals()[cell_df_dict[fea]]
    df_cell_ids = df_cell_ids.merge(df_cell[['Sample']]).drop_duplicates()  # extract 
print('  {} molecular samples with feature and response data'.format(df_cell_ids.shape[0]))

  2373 molecular samples with feature and response data


In [88]:
# (ap)
print(df_cell.shape)
df_cell_ids['Sample'].map(lambda s: True if 'CCLE' in s.split('.') else False).sum()

(15196, 950)


474

In [89]:
df_drug_ids = df_drugs_with_response
for fea in drug_features:
    df_drug = locals()[drug_df_dict[fea]]
    df_drug_ids = df_drug_ids.merge(df_drug[['Drug']]).drop_duplicates()

df_drug_ids = df_drug_ids.merge(df_selected_drugs).drop_duplicates()
print('  {} selected drugs with feature and response data'.format(df_drug_ids.shape[0]))

  46837 selected drugs with feature and response data


In [90]:
print(df_response.shape)
print(df_cell_ids.shape)
print(df_drug_ids.shape)

(31456191, 8)
(2373, 1)
(46837, 5)


In [91]:
# Extract only the relevant rows from `df_response` (where we have both drug features and cell features)
df_response = df_response[df_response['Sample'].isin(df_cell_ids['Sample']) &
                          df_response['Drug1'].isin(df_drug_ids['Drug']) &
                          (df_response['Drug2'].isin(df_drug_ids['Drug']) | df_response['Drug2'].isnull())]
print(df_response.shape)

(25299969, 8)


In [92]:
# Extract the relevant sources
df_response = df_response[df_response['Source'].isin(train_sep_sources + test_sep_sources)]
df_response.reset_index(drop=True, inplace=True)
print(df_response.shape)
df_response[:2]

(7978373, 8)


,Source,Sample,Drug1,Dose1,Drug2,Dose2,Growth,Study
0,CTRP,CTRP.22RV1,CTRP.138,8.309804,NaN,NaN,0.9386,315625
1,CTRP,CTRP.22RV1,CTRP.138,8.013228,NaN,NaN,0.9068,315625


In [93]:
df_sum = utils.summarize_response_data(df_response)
df_sum

,Growth,Sample,Drug1,Drug2,MedianDose
Source,,,,,
ALMANAC.1A,206580,60,101,101,7.000000
ALMANAC.FF,2062098,60,92,71,6.698970
ALMANAC.FG,1293465,60,98,27,6.522879
CTRP,3397103,812,311,0,6.585027
GDSC,1019127,670,213,0,6.505150


In [94]:
df_response = df_response.assign(Group = utils.assign_partition_groups(df_response, partition_by='drug_pair'))
print(df_response.shape)
print(df_response[:2])

Grouped response data by drug_pair: 5587 groups
(7978373, 9)
  Source      Sample     Drug1     Dose1 Drug2  Dose2  Growth   Study  \
0   CTRP  CTRP.22RV1  CTRP.138  8.309804   NaN    NaN  0.9386  315625   
1   CTRP  CTRP.22RV1  CTRP.138  8.013228   NaN    NaN  0.9068  315625   

                 Group  
0  PubChem.CID.7326481  
1  PubChem.CID.7326481  


In [95]:
df_response[['Source', 'Sample', 'Drug1', 'Study', 'Group']].nunique()

Source         5
Sample      1542
Drug1        627
Study     340749
Group       5587
dtype: int64

# build_feature_list()

In [96]:
single = False
input_features = collections.OrderedDict()
feature_shapes = {}

In [97]:
doses = ['dose1', 'dose2'] if not single else ['dose1']
for dose in doses:
    input_features[dose] = 'dose'
    feature_shapes['dose'] = (1,)

print('input_features\n', input_features)
print('\nfeature_shapes\n', feature_shapes)

input_features
 OrderedDict([('dose1', 'dose'), ('dose2', 'dose')])

feature_shapes
 {'dose': (1,)}


In [98]:
encode_response_source = True
if encode_response_source:  # self.encode_response_source
    input_features['response.source'] = 'response.source'
    feature_shapes['response.source'] = (df_source.shape[1] - 1,)  # self.df_source.shape[1]

print('input_features\n', input_features)
print('\nfeature_shapes\n', feature_shapes)

input_features
 OrderedDict([('dose1', 'dose'), ('dose2', 'dose'), ('response.source', 'response.source')])

feature_shapes
 {'dose': (1,), 'response.source': (7,)}


In [99]:
for fea in cell_features:  # self.cell_features
    print(fea)
    feature_type = 'cell.' + fea
    feature_name = 'cell.' + fea
    
    # df_cell = getattr(self, self.cell_df_dict[fea])
    df_cell = locals()[cell_df_dict[fea]]  # returns rnaseq df
    
    input_features[feature_name] = feature_type
    feature_shapes[feature_type] = (df_cell.shape[1] - 1,)
    
print('input_features\n', input_features)
print('\nfeature_shapes\n', feature_shapes)

rnaseq
input_features
 OrderedDict([('dose1', 'dose'), ('dose2', 'dose'), ('response.source', 'response.source'), ('cell.rnaseq', 'cell.rnaseq')])

feature_shapes
 {'dose': (1,), 'response.source': (7,), 'cell.rnaseq': (949,)}


In [100]:
drugs = ['drug1', 'drug2'] if not single else ['drug1']
for drug in drugs:
    for fea in drug_features:  # self.drug_features
        feature_type = 'drug.' + fea
        feature_name = drug + '.' + fea
        
        # df_drug = getattr(self, drug_df_dict[fea])
        df_drug = locals()[drug_df_dict[fea]]
        
        input_features[feature_name] = feature_type
        feature_shapes[feature_type] = (df_drug.shape[1] - 1,)
        
print('input_features\n', input_features)
print('\nfeature_shapes\n', feature_shapes)

input_features
 OrderedDict([('dose1', 'dose'), ('dose2', 'dose'), ('response.source', 'response.source'), ('cell.rnaseq', 'cell.rnaseq'), ('drug1.descriptors', 'drug.descriptors'), ('drug1.fingerprints', 'drug.fingerprints'), ('drug2.descriptors', 'drug.descriptors'), ('drug2.fingerprints', 'drug.fingerprints')])

feature_shapes
 {'dose': (1,), 'response.source': (7,), 'cell.rnaseq': (949,), 'drug.descriptors': (5270,), 'drug.fingerprints': (2048,)}


In [101]:
input_dim = sum([np.prod(feature_shapes[x]) for x in input_features.values()])
print(input_dim)
print(1*2 + 7 + 949 + 5270*2 + 2048*2)

15594
15594


In [102]:
print('Input features shapes:')
for k, v in input_features.items():  # self.input_features.items()
    print('  {}: {}'.format(k, feature_shapes[v]))  # self.feature_shapes[v]
print('Total input dimensions: {}'.format(input_dim))  # self.input_dim

Input features shapes:
  dose1: (1,)
  dose2: (1,)
  response.source: (7,)
  cell.rnaseq: (949,)
  drug1.descriptors: (5270,)
  drug1.fingerprints: (2048,)
  drug2.descriptors: (5270,)
  drug2.fingerprints: (2048,)
Total input dimensions: 15594


# Partition data
loader.partition_data()

In [123]:
val_split = 0.2
train_split = 1 - val_split

In [124]:
partition_by = 'drug_pair'
cv_folds = 1
cell_types = None
by_cell = None
by_drug = None

In [125]:
train_sep_sources = ['CTRP', 'GDSC', 'ALMANAC.FG', 'ALMANAC.FF', 'ALMANAC.1A']
test_sep_sources =  ['CTRP', 'GDSC', 'ALMANAC.FG', 'ALMANAC.FF', 'ALMANAC.1A']

In [126]:
mask = df_response['Source'].isin(train_sep_sources)
test_mask = df_response['Source'].isin(test_sep_sources)  # this will be used for the actual test set (not validation)

In [132]:
# Groups for training
df_group = df_response[mask]['Group'].drop_duplicates().reset_index(drop=True)

In [133]:
df_group.head()

0    PubChem.CID.7326481
1    PubChem.CID.1641662
2    PubChem.CID.2842253
3    PubChem.CID.2729026
4     PubChem.CID.613000
Name: Group, dtype: object

In [134]:
from sklearn.model_selection import KFold, ShuffleSplit
if cv_folds > 1:
    selector = KFold(n_splits=cv_folds, shuffle=True, random_state=SEED)
else:
    selector = ShuffleSplit(n_splits=1, train_size=train_split, test_size=val_split, random_state=SEED)

splits = selector.split(df_group)

In [119]:
# (train_group_index, val_group_index) = next(splits)
# train_groups = set(df_group.values[train_group_index])
# val_groups = set(df_group.values[val_group_index])
# print(len(train_groups))
# print(len(val_groups))

In [137]:
# train_index = df_response.index[df_response['Group'].isin(train_groups) & mask]  # training indices for df_response
# val_index = df_response.index[df_response['Group'].isin(val_groups) & mask]  # val indices for df_response
# test_index = df_response.index[~df_response['Group'].isin(train_groups) & ~df_response['Group'].isin(val_groups) & test_mask]  # test indices for df_response
# print(len(train_index))
# print(len(val_index))
# print(len(test_index))

6437251
1541122
0


In [138]:
train_indexes = []
val_indexes = []
test_indexes = []

for index, (train_group_index, val_group_index) in enumerate(splits):
    train_groups = set(df_group.values[train_group_index])
    val_groups = set(df_group.values[val_group_index])
    
    train_index = df_response.index[df_response['Group'].isin(train_groups) & mask]
    val_index = df_response.index[df_response['Group'].isin(val_groups) & mask]
    
    test_index = df_response.index[~df_response['Group'].isin(train_groups) & ~df_response['Group'].isin(val_groups) & test_mask]

    train_indexes.append(train_index)
    val_indexes.append(val_index)
    test_indexes.append(test_index)

# Build model
build_model()

In [140]:
permanent_dropout=0,
silent=False
dense_feature_layers = [1000, 1000, 1000]

In [141]:
feature_shapes

{'dose': (1,),
 'response.source': (7,),
 'cell.rnaseq': (949,),
 'drug.descriptors': (5270,),
 'drug.fingerprints': (2048,)}

In [ ]:
input_models = {}
dropout_rate = args.drop
for fea_type, shape in feature_shapes.items():
    base_type = fea_type.split('.')[0]
    if base_type in ['cell', 'drug']:
        box = build_feature_model(input_shape=shape,
                                  name=fea_type,
                                  dense_layers=dense_feature_layers,  # args.dense_feature_layers
                                  dropout_rate=dropout_rate,
                                  permanent_dropout=permanent_dropout)
        input_models[fea_type] = box

In [ ]:
inputs = []
encoded_inputs = []
for fea_name, fea_type in input_features.items():
    shape = loader.feature_shapes[fea_type]
    fea_input = Input(shape, name='input.'+fea_name)
    inputs.append(fea_input)
    if fea_type in input_models:
        input_model = input_models[fea_type]
        encoded = input_model(fea_input)
    else:
        encoded = fea_input
    encoded_inputs.append(encoded)

merged = keras.layers.concatenate(encoded_inputs)

h = merged
for i, layer in enumerate(args.dense):
    x = h
    h = Dense(layer, activation=args.activation)(h)
    if dropout_rate > 0:
        if permanent_dropout:
            h = PermanentDropout(dropout_rate)(h)
        else:
            h = Dropout(dropout_rate)(h)
    if args.residual:
        try:
            h = keras.layers.add([h, x])
        except ValueError:
            pass
output = Dense(1)(h)